# 1. Futtatási környezet telepítése



A legtöbb itt található kód még mindig a TensorFlow 1.x környezethez készült, ezért kénytelenek vagyunk ezt használni. A futás során látszani fog, hogy az eljárások már elavultak. Erről még szerencsére jó eredményre fogunk jutni.

Érdemes GPU futtatási környezetet választani, ugaynis a futás így sokszor gyorsabb lesz, mint CPU környezetben. A tanítás lefuthat pár tucat percen belül akár.

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
%cd /content
!git clone --quiet https://github.com/tensorflow/models.git

!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk

!pip install -q Cython contextlib2 pillow lxml matplotlib

!pip install -q pycocotools tf_slim lvis


%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.

import os
os.environ['PYTHONPATH'] += ':/content/models/:/content/models/research/:/content/models/research/slim/'

!python object_detection/builders/model_builder.py

## GTSDB

A tanító képek a GTSDB adatbázisból is érkezhetnek. Itt már annotálva található többszáz kép. 43 különböző jelzőtáblát tartalmaznak. Az annotálás formája ugyanakkor nem jó közvetlenül ide, így először azt át kell alakítani. Az adatbázis a következő linken érhető el:

https://sid.erda.dk/public/archives/ff17dc924eba88d5d01a807357d6614c/FullIJCNN2013.zip

A képek ráadásul kisméretűek, így feltehetően csak rengeteg tanítással fognak működni.


## Ez a repo tartalmazza a GTSDB-t, illetve egy egyszerűbb képgyűjteményt is annotálással.

In [3]:
%cd /content
! git clone https://github.com/gume/VITMMA10_OD.git

/content
Cloning into 'VITMMA10_OD'...
remote: Enumerating objects: 200, done.
remote: Counting objects: 100% (200/200), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 1324 (delta 119), reused 198 (delta 118), pack-reused 1124
Receiving objects: 100% (1324/1324), 337.28 MiB | 20.64 MiB/s, done.
Resolving deltas: 100% (185/185), done.
Checking out files: 100% (1247/1247), done.


(opcionális) A következő sor csak akkor, ha a repo valamiért frissülne a futás közben.

In [ ]:
%cd /content/VITMMA10_OD
! git pull

## Tanító szet előállítása a képekből

A tanításra tfrecord fájlokat használunk, amelyek tartalmazzák az összes képet és annotálást is. Két ilyen recordra van szükségünk, az egyik a tanításhoz (train), a másik az ellenőrzéshez (test). A tanulás közben a train képeken végzett tanulás vissz van ellenőrizve a test képekkel.

A képek mellett már xml formában megtalálható az annotálás. Új képek esetén az labelImg program segítségével készíthető annotálás.
https://github.com/tzutalin/labelImg

Az átalakítás után előáll egy címke fájl is, amiben a képeken szereplő címkék vannak.

A record fáljok megörzik a tartalmukat a művelet során. Véletlen újrafuttatás esetén így duplán lesz benne minden. Ezt elkerülendő érdemes először a fájlokat letörölni.

In [4]:
images_path = '/content/VITMMA10_OD/VITMMA10/images/stop'
tfrecord_path = '/content/'

# Generating these files
images_csv = os.path.join(tfrecord_path, 'vitmma10_stop.csv')
images_train_csv = os.path.join(tfrecord_path, 'vitmma10_stop_train.csv')
images_test_csv = os.path.join(tfrecord_path, 'vitmma10_stop_test.csv')

train_record_fname = os.path.join(tfrecord_path, 'train.tfrecord')
test_record_fname = os.path.join(tfrecord_path, 'test.tfrecord')
label_map_pbtxt_fname = os.path.join(tfrecord_path, 'map.pbtxt')

!python /content/VITMMA10_OD/code/generate_csv.py xml {images_path} {images_csv}
!python /content/VITMMA10_OD/code/generate_pbtxt.py csv {images_csv} {label_map_pbtxt_fname}

import pandas as pd
from sklearn.model_selection import train_test_split

image_df = pd.read_csv(images_csv)
train_files, test_files = train_test_split(image_df.filename.unique(), test_size=0.2)
train_df = pd.DataFrame()
test_df = pd.DataFrame()
for i, row in image_df.iterrows():
  if row.filename in test_files:
    test_df = test_df.append(row)
  else:
    train_df = train_df.append(row)
train_df.to_csv(images_train_csv)
test_df.to_csv(images_test_csv)

!python /content/VITMMA10_OD/code/generate_tfrecord.py {images_train_csv} {label_map_pbtxt_fname} {images_path} {train_record_fname}
!python /content/VITMMA10_OD/code/generate_tfrecord.py {images_test_csv} {label_map_pbtxt_fname} {images_path} {test_record_fname}


100% 74/74 [00:00<00:00, 21910.10it/s]
groups: 100% 59/59 [00:00<00:00, 783.71it/s]
Successfully created the TFRecords: /content/train.tfrecord
groups: 100% 15/15 [00:00<00:00, 781.45it/s]
Successfully created the TFRecords: /content/test.tfrecord


### TFrecord tesztelése

Csak, ha valami formai hiba lenne vele. A képeket meg is lehet jeleníteni, ebben az esetben le kell tölteni a recordokat és egy külön kód meg tudja jeleníteni.

Én ezt a toolt találtam hasznosnak (bár a kód elején érdemes az tensorflow modult tensorflow.compat.v1 -re cserélni). Ez a tool itt a colabon viszont nem futtatható.
https://github.com/sulc/tfrecord-viewer

In [ ]:
import glob
import tensorflow.compat.v1 as tf

total_images = 0
train_files = sorted(glob.glob(os.path.join(tfrecord_path, '*.tfrecord')))
for idx, file in enumerate(train_files):
  try:
    total_images += sum([1 for _ in tf.io.tf_record_iterator(file)]) # Check corrupted tf records
  except Exception as e:
    print("{}: {} is corrupted. {}".format(idx, file, e))
print("Succeed, no corrupted tf records found for {} images".format(total_images))

# 2. Modell kiválasztása és telepítése

Több modell közül is lehet választani. A modellekről itt lehet bővebb információt találni:
https://github.com/tensorflow/models/tree/master/research/object_detection/g3doc

Itt kerül meghatározásra az is, hogy hány lépésből álljon a tanítás. Kis számok esetán (pl. 1000) is lehet már eredményt látni, de a jobb működés érdekében érdemes nagy számmit választani és megvárni a hosszabb tanítást.

In [27]:
# Number of training steps.  # 200000
num_steps = 20000

# Number of evaluation steps.
num_eval_steps = 50

MODELS_CONFIG = {
    'ssd_mobilenet_v2': {
        'model_name': 'ssd_mobilenet_v2_coco_2018_03_29',
        'pipeline_file': 'ssd_mobilenet_v2_coco.config',
        'batch_size': 12
    },
    'ssd_inception_v2': {
        'model_name': 'ssd_inception_v2_coco_2018_01_28',
        'pipeline_file': 'ssd_inception_v2_coco.config',
        'batch_size': 8
    },
    'ssd_mobilenet_v2_quantized': {
        'model_name': 'ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03',
        'pipeline_file': 'ssd_mobilenet_v2_quantized_300x300_coco.config',
        'batch_size': 12
    },
    'ssd_mobilenet_v1_fpn': {
        'model_name': 'ssd_mobilenet_v1_fpn_shared_box_predictor_640x640_coco14_sync_2018_07_03',
        'pipeline_file': 'ssd_mobilenet_v1_fpn_coco.config',
        'batch_size': 12
    },
    'ssd_mobilenet_v1': {
        'model_name': 'ssd_mobilenet_v1_coco_2018_01_28',
        'pipeline_file': 'ssd_mobilenet_v1_coco.config',
        'batch_size': 12
    },
    'faster_rcnn_inception_v2': {
        'model_name': 'faster_rcnn_inception_v2_coco_2018_01_28',
        'pipeline_file': 'faster_rcnn_inception_v2_coco.config',
        'batch_size': 1
    },
    'rfcn_resnet101': {
        'model_name': 'rfcn_resnet101_coco_2018_01_28',
        'pipeline_file': 'rfcn_resnet101_coco.config',
        'batch_size': 8
    },
    'faster_rfcn_resnet101': {
        'model_name': 'faster_rcnn_resnet101_coco_2018_01_28',
        'pipeline_file': 'faster_rfcn_resnet101_coco.config',
        'batch_size': 8
    }
}

# Pick the model you want to use
# Select a model in `MODELS_CONFIG`.
selected_model = 'faster_rcnn_inception_v2'

# Name of the object detection model to use.
MODEL = MODELS_CONFIG[selected_model]['model_name']

# Name of the pipline file in tensorflow object detection API.
pipeline_file = MODELS_CONFIG[selected_model]['pipeline_file']

# Training batch size fits in Colabe's Tesla K80 GPU memory for selected model.
batch_size = MODELS_CONFIG[selected_model]['batch_size']


## Model letöltése


In [7]:
! rm -rf /content/models/research/pretrained_model

In [8]:
%cd /content/models/research

import os
import shutil
import glob
import urllib.request
import tarfile
MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
DEST_DIR = '/content/models/research/pretrained_model'

if not (os.path.exists(MODEL_FILE)):
    urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
    shutil.rmtree(DEST_DIR)
os.rename(MODEL, DEST_DIR)

/content/models/research


In [9]:
!echo {DEST_DIR}
!ls -alh {DEST_DIR}

/content/models/research/pretrained_model
total 111M
drwxr-xr-x  3 345018 5000 4.0K Feb  1  2018 .
drwxr-xr-x 24 root   root 4.0K Nov 28 11:09 ..
-rw-r--r--  1 345018 5000   77 Feb  1  2018 checkpoint
-rw-r--r--  1 345018 5000  55M Feb  1  2018 frozen_inference_graph.pb
-rw-r--r--  1 345018 5000  51M Feb  1  2018 model.ckpt.data-00000-of-00001
-rw-r--r--  1 345018 5000  16K Feb  1  2018 model.ckpt.index
-rw-r--r--  1 345018 5000 5.5M Feb  1  2018 model.ckpt.meta
-rw-r--r--  1 345018 5000 3.2K Feb  1  2018 pipeline.config
drwxr-xr-x  3 345018 5000 4.0K Feb  1  2018 saved_model


## Konfigurációs fájl készítése

A modell felépítése egy konfigurációs fájlban található. A modellel együtt elérhető az eredeti konfiguráció is, amit csak pár helyen elegendő módosítani. Az alábbi kód elvégzi a szükséges módosításokat.

In [10]:
DEST_DIR = "/content/models/research/pretrained_model"
!cp /content/models/research/object_detection/samples/configs/{pipeline_file} {DEST_DIR}

fine_tune_checkpoint = os.path.join(DEST_DIR, "model.ckpt")
print(fine_tune_checkpoint)
print(pipeline_file)

/content/models/research/pretrained_model/model.ckpt
faster_rcnn_inception_v2_coco.config


In [12]:
import os
import re

def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())

pipeline_fname_original = os.path.join(DEST_DIR, pipeline_file)
pipeline_fname = os.path.join(DEST_DIR, "pipeline.config")
assert os.path.isfile(pipeline_fname_original), '`{}` not exist'.format(pipeline_fname_original)

num_classes = get_num_classes(label_map_pbtxt_fname)
with open(pipeline_fname_original) as f:
    s = f.read()
with open(pipeline_fname, 'w') as f:
    
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    # tfrecord files train and test.
    s = re.sub(
        '(input_path: ".*?)(train.record)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(val.record)(.*?")', 'input_path: "{}"'.format(test_record_fname), s)

    # label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
    f.write(s)

Néhány konfigurációs fájl javításra szorulhat. Google keresés segíthet a problémákon.

# 3. Tanítás

In [13]:
%cd /content

model_dir = 'training/'
# Optionally remove content in output model directory to fresh start.
!rm -rf {model_dir}
os.makedirs(model_dir, exist_ok=True)

/content


A tensorboard segítségével figyelemmel kísérhető a tanítás állapota. Itt colab-ban inkább csak egy egy futtatás után látjuk, hogy mi történt. A image tabnál egész jól használható információt kapunk a tanítás állapotáról. A képeknél a bal oldalon látjuk, hogy működik e a felismerés, ha nincs ott bekeretezve semmi, akkor bizony nem működik. 

In [31]:
%reload_ext tensorboard

In [ ]:
%cd /content
%tensorboard --logdir 'training/'

(opcionális) A figyelmeztető üzenetek kikapcsolása

In [16]:
import warnings
warnings.filterwarnings('ignore')

## A konkrét tanítás

In [17]:
pipeline_fname = os.path.join(DEST_DIR, "pipeline.config")
%cd /content

/content


In [28]:
!python /content/models/research/object_detection/model_main.py \
    --pipeline_config_path={pipeline_fname} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --num_eval_steps={num_eval_steps}

W1128 11:36:59.259374 140319373666176 model_lib.py:793] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting train_steps: 20000
I1128 11:36:59.259686 140319373666176 config_util.py:552] Maybe overwriting train_steps: 20000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I1128 11:36:59.259785 140319373666176 config_util.py:552] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: 1
I1128 11:36:59.259863 140319373666176 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: 1
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
I1128 11:36:59.259962 140319373666176 config_util.py:552] Maybe overwriting eval_num_epochs: 1
W1128 11:36:59.260066 140319373666176 model_lib.py:809] Expected number of evaluation epochs is 1, but instead encountered `eval_on_train_input_config.num_epochs` = 0. Overwriting `num_epochs` to 1.
INFO:tensorflow:create_estimator_and_inputs: use_tpu False, export_to

Az alábbi kód egy régebbi módszer a tanításhoz. Elegendő a fenti kód használata erre nincs szükség

In [ ]:
# !python /content/models/research/object_detection/legacy/train.py \
#    --logtostderr \
#    --train_dir={model_dir} \
#    --pipeline_config_path={pipeline_fname}

# 4. Modell konvertálása és elmentése

Amikor már elégedettek vagyunk az eredménnyel, a modellt használható állapotban kell elmenteni. 

In [33]:
!rm -rf '/content/fine_tuned_model'

Ez a konfiguráció csak tesztelés jellegel. Az eredeti modellre vonatkozik, így azt a modellt lehet elmenteni, amiből a tanítás kiindult.

In [34]:
# output_directory = '/content/fine_tuned_model'
# pipeline_fname = '/content/models/research/pretrained_model/pipeline.config.original'
# last_model_path = '/content/models/research/pretrained_model/model.ckpt'
# saved_model_path = '/content/models/research/pretrained_model/saved_model'

Megkeresi a legutolsó tanítási állapotot

In [35]:
output_directory = '/content/fine_tuned_model'
pipeline_fname = '/content/models/research/pretrained_model/pipeline.config'
saved_model_path = ''

import re
import numpy as np

lst = os.listdir(model_dir)
lst = [l for l in lst if 'model.ckpt-' in l and '.meta' in l]
steps=np.array([int(re.findall('\d+', l)[0]) for l in lst])
last_model = lst[steps.argmax()].replace('.meta', '')

last_model_path = os.path.join(model_dir, last_model)
print(last_model_path)


training/model.ckpt-20000


In [36]:
pipeline_fname='/content/models/research/pretrained_model/pipeline.config'

A modell exportálása, ezt a fájl tudjuk majd később használni.

In [ ]:
!python /content/models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path={pipeline_fname} \
    --output_directory={output_directory} \
    --trained_checkpoint_prefix={last_model_path}

## Innentől az átalakítás csak opcionális, különböző formátumokbe lehet konvertálni. Bizonyos környezetekben előnyös lehet egyik másik.

### Átalakítás tflife fromátumba

In [ ]:
!echo "CONVERTING saved model to TF Lite file..."
!tflite_convert \
  --output_file='{output_directory}/model.tflite' \
  --saved_model_dir='{saved_model_path}' \
  --enable_v1_converter


In [ ]:
!echo "CONVERTING saved model to TF Lite file..."
!python /content/models/research/object_detection/export_tflite_ssd_graph.py \
    --pipeline_config_path {pipeline_fname} \
    --trained_checkpoint_prefix {last_model_path} \
    --output_directory {output_directory}

### Átalakítás ONNX fromátumba

In [ ]:
!pip install tf2onnx

In [ ]:
!python -m tf2onnx.convert \
 --saved-model {saved_model_path} \
 --output {output_directory}/frozen_inference_graph.onnx

In [ ]:
!python -m tf2onnx.convert \
  --input {frozen_inference_graph} \
  --inputs 'image_tensor:0[1,300,300,3]' \
  --outputs 'detection_boxes:0,detection_scores:0,num_detections:0,detection_classes:0' \
  --opset 11 --fold_const \
  --output {output_directory}/frozen_inference_graph.onnx

In [ ]:
output_directory = '/content/fine_tuned_model'
pipeline_fname = '/content/fine_tuned_model/pipeline.config'
last_model_path = '/content/fine_tuned_model/model.ckpt'
saved_model_path = '/content/fine_tuned_model/saved_model'
frozen_inference_graph = '/content/fine_tuned_model/frozen_inference_graph.pb'

In [ ]:
!pip install onnx

ONNX formátum tesztelése

In [ ]:
 import onnx

 # Checks 
 onnx_model = onnx.load('/content/fine_tuned_model/frozen_inference_graph.onnx')  # load onnx model 
 onnx.checker.check_model(onnx_model)  # check onnx model 
 print(onnx.helper.printable_graph(onnx_model.graph))  # print a human readable representation of the graph 
 print('ONNX export success, saved as %s\nView with https://github.com/lutzroeder/netron' % f) 

## Modell letöltése

Attól függően milyen formátumba konvertáltuk, különböző fájlokat lehet letölteni. Aalap esetben a következő két fájlra lesz szükség:


In [ ]:
from google.colab import files
files.download(pb_fname)

In [ ]:
from google.colab import files
files.download(label_map_pbtxt_fname)

# 5. Használat

A használat során újabb képekkel tesztelünk

## Tesztelés TensorFlowval

In [42]:
import os
import glob

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = '/content/fine_tuned_model/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = label_map_pbtxt_fname

# If you want to test the code with your images, just add images files to the PATH_TO_TEST_IMAGES_DIR.
#PATH_TO_TEST_IMAGES_DIR =  '/content/VITMMA10_OD/data/images/test' # os.path.join(repo_dir_path, "test")
#
#assert os.path.isfile(pb_fname)
#assert os.path.isfile(PATH_TO_LABELS)
#TEST_IMAGE_PATHS = glob.glob(os.path.join(PATH_TO_TEST_IMAGES_DIR, "*.jpg"))
#assert len(TEST_IMAGE_PATHS) > 0, 'No image found in `{}`.'.format(PATH_TO_TEST_IMAGES_DIR)
#print(TEST_IMAGE_PATHS)
TEST_IMAGE_PATHS = ['/content/street1z.jpg']


In [ ]:
%cd /content/models/research/object_detection

import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

# This is needed to display the images.
%matplotlib inline

from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util


detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')


label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(
    label_map, max_num_classes=num_classes, use_display_name=True)
category_index = label_map_util.create_category_index(categories)


def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)


def run_inference_for_single_image(image, graph):
    with graph.as_default():
        with tf.Session() as sess:
            # Get handles to input and output tensors
            ops = tf.get_default_graph().get_operations()
            all_tensor_names = {
                output.name for op in ops for output in op.outputs}
            tensor_dict = {}
            for key in [
                'num_detections', 'detection_boxes', 'detection_scores',
                'detection_classes', 'detection_masks'
            ]:
                tensor_name = key + ':0'
                if tensor_name in all_tensor_names:
                    tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
                        tensor_name)
            if 'detection_masks' in tensor_dict:
                # The following processing is only for single image
                detection_boxes = tf.squeeze(
                    tensor_dict['detection_boxes'], [0])
                detection_masks = tf.squeeze(
                    tensor_dict['detection_masks'], [0])
                # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
                real_num_detection = tf.cast(
                    tensor_dict['num_detections'][0], tf.int32)
                detection_boxes = tf.slice(detection_boxes, [0, 0], [
                                           real_num_detection, -1])
                detection_masks = tf.slice(detection_masks, [0, 0, 0], [
                                           real_num_detection, -1, -1])
                detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                    detection_masks, detection_boxes, image.shape[0], image.shape[1])
                detection_masks_reframed = tf.cast(
                    tf.greater(detection_masks_reframed, 0.5), tf.uint8)
                # Follow the convention by adding back the batch dimension
                tensor_dict['detection_masks'] = tf.expand_dims(
                    detection_masks_reframed, 0)
            image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

            # Run inference
            output_dict = sess.run(tensor_dict,
                                   feed_dict={image_tensor: np.expand_dims(image, 0)})

            # all outputs are float32 numpy arrays, so convert types as appropriate
            output_dict['num_detections'] = int(
                output_dict['num_detections'][0])
            output_dict['detection_classes'] = output_dict[
                'detection_classes'][0].astype(np.uint8)
            output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
            output_dict['detection_scores'] = output_dict['detection_scores'][0]
            if 'detection_masks' in output_dict:
                output_dict['detection_masks'] = output_dict['detection_masks'][0]
    return output_dict


for image_path in TEST_IMAGE_PATHS:
    image = Image.open(image_path)
    # the array based representation of the image will be used later in order to prepare the
    # result image with boxes and labels on it.
    image_np = load_image_into_numpy_array(image)
    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image_np, axis=0)
    # Actual detection.
    output_dict = run_inference_for_single_image(image_np, detection_graph)
    print(output_dict)
    # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        output_dict['detection_boxes'],
        output_dict['detection_classes'],
        output_dict['detection_scores'],
        category_index,
        instance_masks=output_dict.get('detection_masks'),
        use_normalized_coordinates=True,
        line_thickness=8)
    plt.figure(figsize=IMAGE_SIZE)
    plt.imshow(image_np)

## Tesztelés OPENCVvel

In [ ]:
!pip install opencv-python

In [ ]:
%cd /content
!git clone https://github.com/opencv/opencv.git

In [ ]:
!python /content/opencv/samples/dnn/tf_text_graph_faster_rcnn.py --input {frozen_inference_graph} --config {pipeline_fname} --output /content/opencv.pbtxt

In [ ]:
# How to load a Tensorflow model using OpenCV
# Jean Vitor de Paulo Blog - https://jeanvitor.com/tensorflow-object-detecion-opencv/

import cv2

# Load a model imported from Tensorflow
#tensorflowNet = cv2.dnn.readNetFromTensorflow(pb_fname, label_map_pbtxt_fname)
tensorflowNet = cv2.dnn.readNetFromTensorflow('/content/fine_tuned_model/frozen_inference_graph.pb', '/content/opencv.pbtxt' )
#tensorflowNet = cv2.dnn.readNetFromONNX('/content/fine_tuned_model/frozen_inference_graph_test.onnx')

# Input image
img = cv2.imread('/content/VITMMA10_OD/data/images/train/30_1.jpg')
rows, cols, channels = img.shape

# Use the given image as input, which needs to be blob(s).
tensorflowNet.setInput(cv2.dnn.blobFromImage(img, size=(300, 300), swapRB=True, crop=False))

# Runs a forward pass to compute the net output
networkOutput = tensorflowNet.forward()

# Loop on the outputs
for detection in networkOutput[0,0]:
    
    score = float(detection[2])
    if score > 0.2:
    	
        left = detection[3] * cols
        top = detection[4] * rows
        right = detection[5] * cols
        bottom = detection[6] * rows

        #draw a red rectangle around detected objects
        cv2.rectangle(img, (int(left), int(top)), (int(right), int(bottom)), (0, 0, 255), thickness=2)

# Show the image with a rectagle surrounding the detected objects 
cv2.imshow('Image', img)
cv2.waitKey()
cv2.destroyAllWindows()